https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_10_3_transformer_timeseries.ipynb#scrollTo=hoT-VFSdOANz

In [92]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: cuda


In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [96]:
df = pd.read_csv("../merged.csv", usecols=["INNER_TPRT_1", "EXTN_TPRT", "INNER_HMDT_1"])
df_train = df.iloc[:8000]
df_test = df[8000:]

cols = ['INNER_TPRT_1', "EXTN_TPRT"]

spots_train = df_train[cols].to_numpy().reshape(-1, len(cols))
spots_test = df_test[cols].to_numpy().reshape(-1, len(cols))

spots_train[:5], spots_test[:5]

(array([[15.45, -1.35],
        [15.3 , -1.3 ],
        [15.25, -1.5 ],
        [15.3 , -1.55],
        [15.35, -1.4 ]]),
 array([[13.87,  2.67],
        [13.8 ,  2.57],
        [13.53,  2.5 ],
        [13.83,  2.37],
        [13.5 ,  2.47]]))

In [101]:
# Sequence Data Preparation
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs) - seq_size):
        window = obs[i:(i + seq_size)]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return (
        torch.tensor(x, dtype=torch.float32).view(-1, seq_size, len(cols)),
        torch.tensor(y, dtype=torch.float32).view(-1, len(cols)) 
        # torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), 
        # torch.tensor(y, dtype=torch.float32).view(-1, 1)
    )

x_train, y_train = to_sequences(SEQUENCE_SIZE, spots_train)
x_test, y_test = to_sequences(SEQUENCE_SIZE, spots_test)

x_train.shape, y_train.shape

(torch.Size([7990, 10, 2]), torch.Size([7990, 2]))

In [102]:

# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [117]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()


        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x
    
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, output_size=1, d_model=32, nhead=4, num_layers=1):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

In [118]:
# Train the model
model = TransformerModel(input_dim=len(cols), output_size=len(cols)).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)


In [119]:

epochs = 100
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    # if val_loss < min_val_loss:
    #     min_val_loss = val_loss
    #     early_stop_count = 0
    # else:
    #     early_stop_count += 1

    # if early_stop_count >= 5:
    #     print("Early stopping!")
    #     break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")



Epoch 1/100, Validation Loss: 9.6786
Epoch 2/100, Validation Loss: 3.7340
Epoch 3/100, Validation Loss: 1.3740
Epoch 4/100, Validation Loss: 0.3262
Epoch 5/100, Validation Loss: 0.3994
Epoch 6/100, Validation Loss: 0.7176
Epoch 7/100, Validation Loss: 0.2259
Epoch 8/100, Validation Loss: 0.4459
Epoch 9/100, Validation Loss: 0.1620
Epoch 10/100, Validation Loss: 0.8265
Epoch 11/100, Validation Loss: 0.4845
Epoch 12/100, Validation Loss: 0.5052
Epoch 00013: reducing learning rate of group 0 to 5.0000e-03.
Epoch 13/100, Validation Loss: 0.4477
Epoch 14/100, Validation Loss: 0.1362
Epoch 15/100, Validation Loss: 0.2386
Epoch 16/100, Validation Loss: 0.1272
Epoch 17/100, Validation Loss: 0.1525
Epoch 18/100, Validation Loss: 0.1184
Epoch 19/100, Validation Loss: 0.2491
Epoch 20/100, Validation Loss: 0.1951
Epoch 21/100, Validation Loss: 0.1437
Epoch 00022: reducing learning rate of group 0 to 2.5000e-03.
Epoch 22/100, Validation Loss: 0.1980
Epoch 23/100, Validation Loss: 0.1702
Epoch 24/10